In [1]:
import cv2
import numpy as np
import cv2
import time
from IPython.display import clear_output
import math
from IPython.display import clear_output
    
#WIND_X = 480 
#WIND_Y = 360

WIND_X = 640
WIND_Y = 480

def get_available_cameras (upper_bound = 10, lower_bound = 0):
    available = []
    
    for i in range (lower_bound, upper_bound):
        cap = cv2.VideoCapture (i)
    
        if (cap.isOpened ()):
            available.append (i)
    
        cap.release ()
    
    return available

def detect_marker (img, color):
    #color = [hsv_min, hsv_max]
    
    # преобразуем RGB картинку в HSV модель
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # применяем цветовой фильтр

    thresh = cv2.inRange(hsv, color[0], color[1])
    
    # вычисляем моменты изображения
#    moments = cv2.moments(thresh, 1)
#    dM01 = moments['m01']
#    dM10 = moments['m10']
#    dArea = moments['m00']
    # будем реагировать только на те моменты,
    # которые содержать больше 100 пикселей
    
#    marker_coord = [0, 0]
#    if dArea > 100:
#        x = int(dM10 / dArea)
#        y = int(dM01 / dArea)
#        marker_coord[0] = x
#        marker_coord[1] = y
        #cv2.circle(img, (x, y), 10, (0,0,255), -1)
#    return marker_coord
    
    return thresh

def find_max_bounding_box (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    return (left, top), (left + width, top + height)

def find_low_middle_point (mask):
    result = np.array (mask)
    output = cv2.connectedComponentsWithStats (mask, 8, cv2.CV_32S)
    labels_num = output      [0]
    labels     = output      [1]
    stats      = output      [2]
    sz         = stats.shape [0]
    
    max_w     = 0
    max_label = 0
    
    for label_num in range (1, sz - 1):
        if (stats [label_num, cv2.CC_STAT_WIDTH] > max_w):
            max_w = stats [label_num, cv2.CC_STAT_WIDTH]
            max_label = label_num
    
    top    = stats [max_label, cv2.CC_STAT_TOP]
    left   = stats [max_label, cv2.CC_STAT_LEFT]
    width  = stats [max_label, cv2.CC_STAT_WIDTH]
    height = stats [max_label, cv2.CC_STAT_HEIGHT]
    
    print ("wid", width)
    
    return (left + int (width / 2), top + height)

def find_goal (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 255, 220), 30, 3, use_morph = True)    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (180, 220, 180), 30, 3, use_morph = True)    

    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    tl, br = find_max_bounding_box (goal_mask_res)
    return tl, br

def find_goal_middle (img):
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (245, 245, 245), 40, 3, use_morph = True)    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 15, 1, use_rg = True, use_morph = True)    
    
    #goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 250, 220), 30, 3, use_morph = True)    
    
    goal1, g_mask1 = obtain_color_ratio_mask (img, (220, 235, 220), 20, 3, use_morph = True)    
    
    #goal2, g_mask2 = obtain_color_ratio_mask (img, (85, 85, 85), 13, 1, use_rg = True, use_morph = True)    
    
    #cv2.imshow ("kek", goal1)
    #cv2.imshow ("kek2", goal2)
    
    #goal_mask_res = cv2.bitwise_and (g_mask1, g_mask2)
    #goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)

    goal_mask_res = g_mask1
    goal_res = cv2.bitwise_and (img, img, mask = goal_mask_res)
    
    mid = find_low_middle_point (goal_mask_res)
    return mid

#адаптивный трешхолд
#функция, на вход картинка на выход координаты середины
#написать, что надо застелить окрестность ворот
#

In [2]:
#"""
cam = cv2.VideoCapture (0)
#cam = cv2.VideoCapture ("inp.webm")

#cv2.namedWindow  ("field", cv2.WINDOW_NORMAL)
#cv2.resizeWindow ("field", (WIND_X, WIND_Y))

output_shape = (WIND_X * 3, WIND_Y * 2)

#fourcc = cv2.VideoWriter_fourcc (*'XVID')
#out    = cv2.VideoWriter ('output.avi', fourcc, 20.0, output_shape)
#out    = cv2.VideoWriter ('output.mp4', fourcc, 20.0, output_shape)

cv2.namedWindow  ("goal detection", cv2.WINDOW_NORMAL)
cv2.resizeWindow ("goal detection", output_shape)

marker_color_min = (60, 130, 80)
marker_color_max = (100, 170, 120)

while (cam.isOpened ()):
    ret1, frame = cam.read ()
    
    if (ret1 != True):
        break
    
    frame_resized = cv2.resize (frame, (int (WIND_X), int (WIND_Y)))
    
    cv2.waitKey (1)
    
    #tl, br = find_goal (frame_resized)
    
    th = detect_marker (frame_resized, (marker_color_min, marker_color_max))
    
    tl, br = find_max_bounding_box (th)
    
    goal_on_inp = cv2.rectangle (frame_resized.copy (), tl, br, (255, 0, 0), 5)
    
    ###mid = find_goal_middle (frame_resized)
    ###goal_on_inp_ = cv2.circle (frame_resized.copy (), mid, 5, (250, 10, 253), -1)
    
    goal_marked = cv2.addWeighted (frame_resized, 0.6, goal_on_inp, 0.4, 0)
    ###goal_marked = cv2.addWeighted (goal_marked, 0.6, goal_on_inp_, 0.4, 0)
    
    #output_image = np.concatenate ((frame_resized, goal_res), axis = 1)
    output_image = np.concatenate ((frame_resized, goal_marked), axis = 1)
    
    cv2.imshow ("goal detection", output_image)
    #out.write (output_image)
    
    #x0 =  1        #x coordinate of the point on the screen, pixels
    #y0 = -1.3      #y coordinate of the point on the screen, pixels
    
    #x0 = (tl [0] + br [0]) / 2 - WIND_X / 2
    #y0 = - tl [1] + WIND_Y / 2
    #k1 =  0      #incline  angle.               from  0 t0 1
    #k2 =  0      #rotation around Z axis angle. from -1 to 1
    
    #A = point_position(k1, k2, x0, y0)
    #B = ball_position(k1, k2, x0, y0)
    #clear_output (wait = True)
    #print (A)
    #print (x0, y0)
    
    time.sleep  (0.01)
    
    if (cv2.waitKey (1) & 0xFF == ord ('q')):
        break

cam.release ()
#out.release ()

cv2.waitKey           (1)
cv2.destroyAllWindows ()
#"""